In [1]:
import requests
import pandas as pd
import numpy as np

output_json = []

# setup search parameters
process = 'met:Ingest'
datastream = 'met.00'

# API link formatted with search parameter
url = "https://engineering.arm.gov/dsdb/cgi-bin/procdb?id={}&action=get-context&prod=1".format(datastream)

response = requests.get(url)
contents = response.text

#Separates the scripts into an array of lines
lines = contents.splitlines()[1:-1]
# cut first and last lines with content: '<graphml>' and '</graphml>'

#Creates the lists and add to them the lines corresponding to nodes and edges

node_list = []
edge_list = []
for line in lines:
    if('node id=' in line):
        node_list.append(line[8:len(line)-2])
    else:
        edge_list.append(line[8:len(line)-2])

identifier = []
name = []
subt = []
cons = []
prov = []
pos = []
prod = []

#Extract every single part of the data in graphml
for i in range(len(node_list)):
    tmp = node_list[i].replace('"','').split('prod=')
    prod.append(tmp[1])

    tmp = tmp[0].split('pos=')
    pos.append(tmp[1])

    tmp = tmp[0].split('prov=')
    prov.append(tmp[1])

    tmp = tmp[0].split('cons=')
    cons.append(tmp[1])

    tmp = tmp[0].split('subt=')
    subt.append(tmp[1])

    tmp = tmp[0].split('name=')
    name.append(tmp[1])

    tmp = tmp[0].split('id=')
    identifier.append(tmp[1].replace(' ',''))

#Create a dataframe with the info extracted from the graphml script for NODES
nodes_df =pd.DataFrame([],columns=['id','name','subt','cons','fields','pos','prod'])
for i in range(0,len(identifier)):
    nodes_df.loc[len(nodes_df)] = [identifier[i],name[i],subt[i],cons[i],prov[i],pos[i],prod[i]]
nodes_df.set_index('id', inplace = True)

#Structure to keep the edges structure
edge_id = []
edge_source = []
edge_target = []

for i in range(len(edge_list)):
    #Some special editing to adapt and format the script 
    edge_list[i] = edge_list[i].replace('"','')
    tmp = edge_list[i].split('target=')
    edge_target.append(tmp[1].replace('\n',''))

    tmp = tmp[0].split('source=')
    edge_source.append(tmp[1].replace(' ',''))

    tmp = tmp[0].split('id=')
    edge_id.append(tmp[1].replace(' ',''))

#Create a dataframe with the info extracted from the graphml script for EDGES
edges_df =pd.DataFrame ([],columns=['id','source','target'])
for i in range(0,len(edge_id)):
    edges_df.loc[len(edges_df)] = [edge_id[i], edge_source[i], edge_target[i]]
edges_df.set_index('id', inplace = True)



#Created a parenhood df to keep track of the hierarchy in the Flowchart
parenthood_df = pd.DataFrame([],columns=['node','parents','children'],dtype=str)
parenthood_df['node'] = pd.Series(nodes_df.index.values)
tmp = np.repeat('',len(nodes_df))
parenthood_df['parents'] = pd.Series(tmp)
parenthood_df['children'] = pd.Series(tmp)
parenthood_df = parenthood_df.set_index('node')
parenthood_df = parenthood_df.astype(str)


def add_parent(node, parent):
    global parenthood_df
    #Get the parents value for the given node 
    parents = parenthood_df.loc[node][0]
    parents = list(parents)
    #Join the old parents with the new one in a list
    parents.extend([parent])
    #Set the new list of parents
    parenthood_df.loc[node, 'parents'] = parents

def add_children(node, child):
    global parenthood_df
    #Get the children value for the given node 
    children = parenthood_df.loc[node][1]
    children = list(children)
    #Join the old parents with the new one in a list
    children.extend([child])
    #Set the new list of parents
    parenthood_df.loc[node, 'children'] = children

for item in edges_df.values:
    add_parent(item[1],item[0])
    add_children(item[0],item[1])
#     print('Parenthood Concluded')

In [ ]:
contents

In [2]:
nodes_df

,name,subt,cons,fields,pos,prod
id,,,,,,
gswcorrbrs1dutt.c1,gswcorrbrs1dutt.c1,,,,right,
mergesonde1mace.c1,mergesonde1mace.c1,,,,right,
qcrad1long.c2,qcrad1long.c2,,,,right,
surfspecalb_elialb:VAP,surfspecalb_elialb : VAP,Uses 56 source field(s),surfspecalb1mlawer.c0 : be_hemisp_broadband_mf...,,right,
qcrad:VAP,qcrad : VAP,Uses 6 source field(s),"met.b1 : atmos_pressure, met.b1 : precip_rate_...",,right,
tqcrad1long_brs:Bundle,tqcrad1long_brs : Bundle,,,,right,
kazrcorpr.c0,kazrcorpr.c0,,,,right,
microbasepi2.c1,microbasepi2.c1,,,,right,
met60s.b1,met60s.b1,,,,right,


In [6]:
procs = edges_df.query("source == 'met.00'")['target']
print("procs: source == 'met.00'\n{}\n".format(procs))
for proc in procs:
    print("source = {}".format(proc))
    targets = edges_df.query("source == '{}'".format(proc))["target"]
    for target in targets:
        if "Ingest" in proc and target[-2] != "0":
            print("\t{} = target".format(target))

procs: source == 'met.00'
id
e6     met:Ingest
e9     met:Bundle
e10    met:Bundle
e12    met:Ingest
Name: target, dtype: object

source = met:Ingest
	met.b1 = target
	twr.b1 = target
	sonicwind2d.b1 = target
	met60s.b1 = target
source = met:Bundle
source = met:Bundle
source = met:Ingest
	met.b1 = target
	twr.b1 = target
	sonicwind2d.b1 = target
	met60s.b1 = target


In [11]:
# coding: utf-8

#Clean all environment variables
from IPython import get_ipython
get_ipython().magic('reset -sf')
from IPython import get_ipython
def __reset__(): get_ipython().magic('reset -sf')

import os
import json
import requests
import urllib.request
import requests

import numpy as np
import pandas as pd 
import glob
import datetime as dt
import time
import psycopg2

In [12]:
 #-----------------------SETTING UP DB ACCESS--------------------------#  
#Set up Database connection
# conn_string = "host='armdev-pgdb.ornl.gov' dbname='arm_all' user='???' password='?????'"
# # print the connection string we will use to connect
# print ("Connecting to database\n->%s" % (conn_string))

# # get a connection, if a connect cannot be made an exception will be raised here
# conn = psycopg2.connect(conn_string)

# # conn.cursor will return a cursor object
# cursor = conn.cursor()


#-----------------------SETTING UP WORKING DIRECTORY--------------------------#
def Working_env(path):
    os.chdir(path)
    os.path.abspath(path)
    
# SET ENVIRONMENT TO THE CONFIG FILE
path = '/Users/ofg/projects/graphml_parse'
Working_env(path)    

#Loads the config file with the processes and dates
f = open("Process_Dates.json")
data = json.load(f)
f.close()

processes = (data.get("process"))
dates = (data.get("dates"))

input_dates_flags = []
# for process in processes:

input_dates = dates
input_dates = sorted(input_dates)

FileNotFoundError: [Errno 2] No such file or directory: 'Process_Dates.json'

In [13]:
output_json = []

process = 'met:Ingest'
datastream = 'met.00'
url = "https://engineering.arm.gov/dsdb/cgi-bin/procdb?id={}&action=get-context&prod=1".format(datastream)
#request = urllib.request.Request(url)
#response = urllib.request.urlopen(request)
response = requests.get(url)
contents = response.text
#Separates the scripts into an array of lines
lines = contents.splitlines()
#Remove the first line with the content: <graphml>
lines.pop(0)
#Remove the last two lines with the content: </graphml>adi_lassocbh:VAP" target="adilassocbh.c1"/> 
                                         #   </graphml>
lines.pop(len(lines)-1)

#Creates the lists and add to them the lines corresponding to nodes and edges

node_list = []
edge_list = []
for line in lines:
    if('node id=' in line):
        node_list.append(line[8:len(line)-2])
    else:
        edge_list.append(line[8:len(line)-2])

identifier = []
name = []
subt = []
cons = []
prov = []
pos = []
prod = []

#Extract every single part of the data in graphml
for i in range(len(node_list)):
    tmp = node_list[i].replace('"','').split('prod=')
    prod.append(tmp[1])

    tmp = tmp[0].split('pos=')
    pos.append(tmp[1])

    tmp = tmp[0].split('prov=')
    prov.append(tmp[1])

    tmp = tmp[0].split('cons=')
    cons.append(tmp[1])

    tmp = tmp[0].split('subt=')
    subt.append(tmp[1])

    tmp = tmp[0].split('name=')
    name.append(tmp[1])

    tmp = tmp[0].split('id=')
    identifier.append(tmp[1].replace(' ',''))

#Create a dataframe with the info extracted from the graphml script for NODES
nodes_df =pd.DataFrame([],columns=['id','name','subt','cons','fields','pos','prod'])
for i in range(0,len(identifier)):
    nodes_df.loc[len(nodes_df)] = [identifier[i],name[i],subt[i],cons[i],prov[i],pos[i],prod[i]]
nodes_df.set_index('id', inplace = True)

#Structure to keep the edges structure
edge_id = []
edge_source = []
edge_target = []

for i in range(len(edge_list)):
    #Some special editing to adapt and format the script 
    edge_list[i] = edge_list[i].replace('"','')
    tmp = edge_list[i].split('target=')
    edge_target.append(tmp[1].replace('\n',''))

    tmp = tmp[0].split('source=')
    edge_source.append(tmp[1].replace(' ',''))

    tmp = tmp[0].split('id=')
    edge_id.append(tmp[1].replace(' ',''))

#Create a dataframe with the info extracted from the graphml script for EDGES
edges_df =pd.DataFrame ([],columns=['id','source','target'])
for i in range(0,len(edge_id)):
    edges_df.loc[len(edges_df)] = [edge_id[i], edge_source[i], edge_target[i]]
edges_df.set_index('id', inplace = True)



#Created a parenhood df to keep track of the hierarchy in the Flowchart
parenthood_df = pd.DataFrame([],columns=['node','parents','children'],dtype=str)
parenthood_df['node'] = pd.Series(nodes_df.index.values)
tmp = np.repeat('',len(nodes_df))
parenthood_df['parents'] = pd.Series(tmp)
parenthood_df['children'] = pd.Series(tmp)
parenthood_df = parenthood_df.set_index('node')
parenthood_df = parenthood_df.astype(str)


def add_parent(node, parent):
    global parenthood_df
    #Get the parents value for the given node 
    parents = parenthood_df.loc[node][0]
    parents = list(parents)
    #Join the old parents with the new one in a list
    parents.extend([parent])
    #Set the new list of parents
    parenthood_df.loc[node, 'parents'] = parents

def add_children(node, child):
    global parenthood_df
    #Get the children value for the given node 
    children = parenthood_df.loc[node][1]
    children = list(children)
    #Join the old parents with the new one in a list
    children.extend([child])
    #Set the new list of parents
    parenthood_df.loc[node, 'children'] = children

for item in edges_df.values:
    add_parent(item[1],item[0])
    add_children(item[0],item[1])
#     print('Parenthood Concluded')

In [21]:
contents

'<graphml>\n  <node id="gswcorrbrs1dutt.c1" name="gswcorrbrs1dutt.c1" subt="" cons="" prov="" pos="right" prod="" />\n  <node id="mergesonde1mace.c1" name="mergesonde1mace.c1" subt="" cons="" prov="" pos="right" prod="" />\n  <node id="qcrad1long.c2" name="qcrad1long.c2" subt="" cons="" prov="" pos="right" prod="" />\n  <node id="surfspecalb_elialb:VAP" name="surfspecalb_elialb : VAP" subt="Uses 56 source field(s)" cons="surfspecalb1mlawer.c0 : be_hemisp_broadband_mfrsr, surfspecalb1mlawer.c0 : be_hemisp_broadband_status_mfrsr, surfspecalb1mlawer.c0 : be_hemisp_narrowband_mfrsr, surfspecalb1mlawer.c0 : be_hemisp_narrowband_status_mfrsr, surfspecalb1mlawer.c0 : be_surface_albedo_mfr_broadband_10m, surfspecalb1mlawer.c0 : be_surface_albedo_mfr_broadband_10m_status, surfspecalb1mlawer.c0 : be_surface_albedo_mfr_broadband_25m, surfspecalb1mlawer.c0 : be_surface_albedo_mfr_broadband_25m_status, surfspecalb1mlawer.c0 : be_surface_albedo_mfr_narrowband_10m, surfspecalb1mlawer.c0 : be_surface_

In [14]:
nodes_df

,name,subt,cons,fields,pos,prod
id,,,,,,
gswcorrbrs1dutt.c1,gswcorrbrs1dutt.c1,,,,right,
mergesonde1mace.c1,mergesonde1mace.c1,,,,right,
qcrad1long.c2,qcrad1long.c2,,,,right,
surfspecalb_elialb:VAP,surfspecalb_elialb : VAP,Uses 56 source field(s),surfspecalb1mlawer.c0 : be_hemisp_broadband_mf...,,right,
qcrad:VAP,qcrad : VAP,Uses 6 source field(s),"met.b1 : atmos_pressure, met.b1 : precip_rate_...",,right,
tqcrad1long_brs:Bundle,tqcrad1long_brs : Bundle,,,,right,
kazrcorpr.c0,kazrcorpr.c0,,,,right,
microbasepi2.c1,microbasepi2.c1,,,,right,
met60s.b1,met60s.b1,,,,right,


In [15]:
edges_df

,source,target
id,,
e1,mettwr:Ingest,met.00
e2,met:Collection,met.00
e3,mettiptwr:Ingest,met.00
e4,mettwr:Collection,met.00
e5,met:Ingest,met.00
e6,met.00,met:Ingest
e7,mettiptwr:Collection,met.00
e8,met:Bundle,met.00
e9,met.00,met:Bundle


In [25]:
parenthood_df.loc['met.00']

parents     [mettwr:Ingest, met:Collection, mettiptwr:Inge...
children     [met:Ingest, met:Bundle, met:Bundle, met:Ingest]
Name: met.00, dtype: object

In [23]:
parenthood_df.loc['met:Ingest']

parents                                      [met.00, met.00]
children    [met.00, met.b1, met.00, twr.b1, sonicwind2d.b...
Name: met:Ingest, dtype: object

In [ ]:
parenthood_df.loc['met.00']['children']

In [ ]:
    #Locate the interested process [Process:VAP] to get its parents and children  
    process_found = False
    while (process_found==False):
        for node in parenthood_df.index.values:
            if process in node:
                if ':VAP' in node:
                    inner_process = node
                    process_found = True

    predecessors = parenthood_df.loc[inner_process][0]
    successors = parenthood_df.loc[inner_process][1]

    # #Examples - lassorl
    # predecessors = ['mplps.a0','sondewnpn.a1', 'arsclkazr1kollias.c0', 'smet60s.b1','interpolatedsonde.c1','kazrcormd.c0']
    # successors = ['smet60s.b1', 'marinemet2.b1', 'aeriprof3feltz.c1']

    # #Examples - lassocloud
    # input_dates = ['2017-09-12','2016-10-17', '2018-03-05', '2013-10-23','2015-10-23', '2004-11-16']
    # predecessors = ['rlprofmerge1turn.c0','1twrmr.c1', 'sgpabrfcpcpqpeX1.c1', 'rllog.a0']
    # successors = ['smet60s.b1', 'marinemet.b1', 'cloudfraction05m.c0']




    #Creates the query structures to query the database looking for the files of the
    # datastreams in the input dates
    def Create_queries(input_dates, ds):
        dates_conditional_query = " AND ("
        for date in input_dates:
            dates_conditional_query += " start_time LIKE '"+date+"%' OR"
        dates_conditional_query = dates_conditional_query[0:len(dates_conditional_query)-3]
        dates_conditional_query += ")"

        datastreams_conditional_query = "SELECT old_filename FROM user_access_files.zebfile_timeplat WHERE ("
        for datastream in ds:
            ds_data,dataLevel = datastream.split('.')
            datastreams_conditional_query += " (zeb_platform LIKE '%"+ds_data +"%"+dataLevel+ "') OR"
        datastreams_conditional_query = datastreams_conditional_query[0:len(datastreams_conditional_query)-3]
        datastreams_conditional_query += ")"

        #Join the queries
        query = ""+datastreams_conditional_query+dates_conditional_query+""
#         print('\n\n',query)
        return query


    #Run the query
    def Run_query(query):
        cursor.execute(query)
        records = cursor.fetchall()
        return records


    #Function to separate the date from the files given the file name
    def Get_date_and_names(files):
        dates = []
        for file in files:
            file = str(file)
            date_temp = file.split(".")[2]
            year = (date_temp[:4])
            month = (date_temp[4:6])
            day = (date_temp[6:8])
            dates.append((str(year) + '-' + str(month) + '-' + str(day)))

        file_name = []
        for file in files:
            file = str(file)
            file = file.replace('(','').replace(',)','').replace("'","")
            name = file.split(".")
            name = name[0]+'.'+name[1]
            #Removes the first 3 letters as they indicate the site and the input nodes has no site info
            file_name.append(name[3:])
        return file_name,dates

    def Create_ds_df(records):
        #Separates the files names from its dates 
        names,dates = Get_date_and_names(records)
        #Create a data frame with these info 
        df = pd.DataFrame([],columns=['File','Date'])
        #predecessors_df.set_index('File', inplace = True)
        df['File'] = names
        df['Date'] = dates
        return df


    #Agregates a set of different dates given that many files in the same day had n different versions
    # then I had to aggregate them into all different dates where we have the files of interest, regardless its version

    # The output goal here was to create a dataframe where I have a datastream column and a date column
    # with all the date from which we have a correspondent file
    def Aggregates_ds(datastreams,df):
        aggregrations = df.groupby('File')['Date']
        keys = aggregrations.groups.keys()
        df_aggregated = pd.DataFrame([],columns=['Datastream','Dates'])
        for key in keys:
            dates = aggregrations.get_group(key)
            for datastream in datastreams:
                ds = datastream.split('.')
                if (ds[0] in key and ds[1] in key):
                    key = datastream
            df_aggregated.loc[len(df_aggregated)] = [key,sorted(list(set(dates.values)))]
        df_aggregated.set_index('Datastream', inplace=True)  
        return df_aggregated


    #Structure to verify if the file for the given data exists and set a flag marker to green in case it exists, 
    # red otherwise
    # The output is list composed by formated dictionaries storing the appropriate info

    def Build_ds_dict(input_dates,ds,df_aggregated):
        datastreams_dates_dict = []
        for datastream in ds:
            #print('DS:',datastream)
            dates_dict = []
            keys = ['Date','Flag']

            ds_flag = 'Green'
            for date in input_dates:
                flag = 'Red'
                list_of_dates = []
                try:
                    list_of_dates = df_aggregated.loc[datastream][0]
                except:
                    pass
                if date in list_of_dates:
                    flag = 'Green'
                else:
                    ds_flag = 'Red'

                values = [date,flag]
                _dict = (dict(zip(keys,values)))
                dates_dict.append(_dict)


            keys = ['Datastreams','Dates','Flag']
            values = [datastream,dates_dict,ds_flag]
            _dict = (dict(zip(keys,values)))
            #print(_dict)
            datastreams_dates_dict.append(_dict)
        return (datastreams_dates_dict)


    #Parents - predecessors
    query = Create_queries(input_dates,predecessors)
    records = Run_query(query)
    predecessors_df = Create_ds_df(records)
    predecessors_df_aggregated = Aggregates_ds(predecessors,predecessors_df)
    input_datastreams_dates_dict = Build_ds_dict(input_dates,predecessors,predecessors_df_aggregated)



    #Children - Successors
    query = Create_queries(input_dates,successors)
    records = Run_query(query)
    successors_df = Create_ds_df(records)
    successors_df_aggregated = Aggregates_ds(successors,successors_df)
    output_datastreams_dates_dict = Build_ds_dict(input_dates,successors,successors_df_aggregated)


    # Continue to generalize to output and finish json formating
    input_output_dict = {"input":input_datastreams_dates_dict,"output":output_datastreams_dates_dict}
    process_flag = 'Green'
    input_dates_flags_aux = 'Green'
    for key in input_output_dict.keys():
        puts = input_output_dict.get(key)
        for ds in puts:
            flag = ds.get('Flag')
            if(flag=='Red'):
                process_flag = 'Red'
                # To flag date
                input_dates_flags_aux = 'Red'

    process_dict = {"process":process,"input_output":input_output_dict, "Flag":process_flag}
    process_dict = json.dumps(process_dict)
    process_dict = json.loads(process_dict)
    output_json.append(process_dict)
    
    input_dates_flags.append(input_dates_flags_aux)
    print(process," FINISHED\n")

In [ ]:
path = '/data/w0l/Backup/Codes/Lasso Project'
Working_env(path)    

dates_dict = (dict(zip(input_dates,input_dates_flags)))
output_json = {"dates":dates_dict,"processes":output_json}
with open('ui.json', 'w') as outfile:
    json.dump(output_json, outfile)
    